In [351]:
import string
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(1234)

In [342]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘robert_frost.txt’ already there; not retrieving.



In [343]:
file = '/content/robert_frost.txt'
lis_line = []

for line in open(file):
    line = line.lower().rstrip()
    if line:
      line = line.translate(str.maketrans('', '', string.punctuation))
      lis_line.append(line.split())



In [344]:
#define fuction to see if a word exists in a dictionary, and either add word and set value to 1 or add 1 to value
def func_add_prob(dict_, word):
  if word not in dict_:
    dict_[word] = 1
  elif word in dict_:
    dict_[word] += 1

#define function tp generate an empty dictionary for word not in a dictionary
def func_add_word(dict_, word):
  if word not in dict_:
    dict_[word] = {}

#define function to convert frequency to probability for a dictionary.
def func_conv_to_prob(dict_):
  sum_ = sum(np.array(list(dict_.values())))
  for key, value in dict_.items():
    dict_[key] = value/sum_

#define function to convert to cumulative probability.
def func_conv_to_cumu(dict_):
  c = 0
  for key, value in dict_.items():
    dict_[key] += c
    c = dict_[key]

#define function to 
def func_add_word_sent(dict_next_word_, sent_, p):
  for next_word, prob in dict_next_word_.items():
    if prob > p and next_word != "End":
      sent_.append(next_word)
      break
    elif prob > p and next_word == "End":
      break
  return(next_word)


In [345]:
#generate empty dictionaries
dict_pi = {}
dict_A1 = {}
dict_A2 = {}

#populate dictionaries
for S in lis_line:
  L = len(S)
  for idx in range(L):
    
    if idx == 0: # initial state distribution
      func_add_prob(dict_pi, S[idx])

    elif idx == 1: #first order for second word
      func_add_word(dict_A1, S[idx-1])
      func_add_prob(dict_A1[S[idx-1]], S[idx])

    elif idx > 1: #second order all words
      func_add_word(dict_A2, (S[idx-2],S[idx-1]))
      func_add_prob(dict_A2[(S[idx-2],S[idx-1])], S[idx])
        
    if idx == L - 1 and idx > 1: #last word in sentence
      func_add_word(dict_A2, (S[idx-1],S[idx]))
      func_add_prob(dict_A2[(S[idx-1],S[idx])], "End")

In [346]:
#calculate probabilities from frequencies
#initial state distribution
func_conv_to_prob(dict_pi)

#first order second word
for key in dict_A1.keys():
  func_conv_to_prob(dict_A1[key])

#second order
for key in dict_A2.keys():
  func_conv_to_prob(dict_A2[key])

In [347]:
#calculate cumulative probabilities from probabilities
#initial state distribution
func_conv_to_cumu(dict_pi)

#first order second word
for key in dict_A1.keys():
  func_conv_to_cumu(dict_A1[key])

#second order
for key in dict_A2.keys():
  func_conv_to_cumu(dict_A2[key])

In [356]:
for lines in range(4):
  sentence = []

  #Generate first word
  p_1 = np.random.random()

  for word, value in dict_pi.items():
    if value > p_1:
      sentence.append(word)
      break

  #Generate secon word
  p_2 = np.random.random()

  for word, value in dict_A1[word].items():
    if value > p_2:
      sentence.append(word)
      break

  #Generate remainder of sentence
  next_word = ""

  while next_word != "End":
    dict_next_word = dict_A2[tuple(sentence[-2:])]
    p = np.random.random()
    next_word = func_add_word_sent(dict_next_word, sentence, p)

  print(" ".join(sentence))

first theres the childrens house of makebelieve
are there they said it and besides
and this bill
my french indian esquimaux
